In [1]:
import os

import numpy as np
import pandas as pd
import xlrd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
stocks = pd.read_csv("/content/drive/MyDrive/Fiverr/PROJECT_MIMI/Stocks_DS_tab_delimited.dat", sep="\t")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,4,23,24,25,26,27,28,29,30,31,32,34,35,38,39,45,66,68,69,72) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
fundamentals = pd.read_csv("/content/drive/MyDrive/Fiverr/PROJECT_MIMI/Fundamentals_Full.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (28,850,892,893,894,895,896,897,898,899,900,901,902,903,905,906,907,908,1747,1748,1749,1750,1754,1758,1760,1761,1764) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
ratings = pd.read_csv("/content/drive/MyDrive/Fiverr/PROJECT_MIMI/Ratings_Full.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (13,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
securities = pd.read_csv("/content/drive/MyDrive/Fiverr/PROJECT_MIMI/Securities_Full.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (42,47,48,49,50,52,53) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
settlements = pd.read_excel("/content/drive/MyDrive/Fiverr/PROJECT_MIMI/Settlements.xlsx", na_values="#NULL!")

In [8]:
def excel_to_dictionary(sheetname, filename="/content/drive/MyDrive/Fiverr/PROJECT_MIMI/DataDictionary.xlsx"):
    abbreviation_dict = dict()
    
    abbreviation = xlrd.open_workbook(os.path.abspath(filename))
    abbreviation_sheet = abbreviation.sheet_by_name(sheetname)
    
    for i in range(1, abbreviation_sheet.nrows):
        abb = abbreviation_sheet.cell(i, 0).value
        abb_definition = abbreviation_sheet.cell(i, 1).value
        abbreviation_dict[abb] = abb_definition
        
    return abbreviation_dict

In [9]:
stocks_dict = excel_to_dictionary(sheetname="Stocks")
fundamentals_dict = excel_to_dictionary(sheetname="Fundamentals")
ratings_dict = excel_to_dictionary(sheetname="Ratings")
securities_dict = excel_to_dictionary(sheetname="Securities")

In [10]:
def rename_columns(df, df_dict):
  df.rename(columns=df_dict, inplace=True)
  df.columns = [column.lower().replace(" ", "_") for column in df.columns]
  return df

In [11]:
stocks = rename_columns(stocks, stocks_dict)
fundamentals = rename_columns(fundamentals, fundamentals_dict)
ratings = rename_columns(ratings, ratings_dict)
securities = rename_columns(securities, securities_dict)